In [ ]:
#default_exp Census_Bureau_urban_areas

In [1]:
# From https://www2.census.gov/geo/pdfs/reference/GARM/Ch12GARM.pdf:

# "Urbanized Areas (UAs)
# A UA is a continuously built-up area with a population of 50,000 or more.
# It comprises one or more places—central place(s)—and the adjacent densely settled surrounding 
# area—urban fringe—consisting of other places and nonplace territory."

# "Urban Places Outside of UAs
# Outside of UAs, an urban place is any incorporated place or census designated place (CDP) with 
# at least 2,500 inhabitants. A CDP is a densely settled population center that has a name and 
# community identity, and is not part of any incorporated place."

# "Rural Places and Territory
# Territory, population, and housing units that the Census Bureau does not classify as urban are classified 
# as rural."

# Census Urban Areas and Urban Clusters are not based on political boundaries at all.

In [1]:
import pandas as pd
import numpy as np

In [3]:
%%time
# (Takes about 1.25 clock minutes.)
year = 2017
infile = 'data/df_%d_OMB.csv' % year
df = pd.read_csv(infile)

df['CBSA Code'] = df['CBSA Code'].fillna(99999)
df['CBSA Code'] = df['CBSA Code'].astype(int)

CPU times: user 1min 17s, sys: 11.4 s, total: 1min 29s
Wall time: 1min 28s


In [4]:
# To use a df copy instead of reading it in again. Uncomment the useful line.
#df_copy = df.copy()
#df = df_copy.copy()

In [5]:
print(len(df[df['CBSA Code']==0]))
print(len(df[df['CBSA Code']==99999]))

883453
4


In [6]:
len(df['CBSA Code'].drop_duplicates())

935

In [2]:
# Add the Urban Area (UA) to each row by matching InfoGroup 'CBSA Code' (metro/micropolitan area) to the Census
# Relationship file that cross-references CBSA and UA.
relationship = 'Relationship_Files/Urban_Area_to_Metro_Micro_Area_utf-8.csv'
rel = pd.read_csv(relationship)

In [11]:
cbsa_list = rel['CBSA'].drop_duplicates().tolist()
ua_list = rel['UA'].drop_duplicates().tolist()
cbsa_list.remove(99999)
ua_list.remove(99999)
print(len(cbsa_list))
print(len(ua_list))

955
3601


In [8]:
# The file contains a record for all metro/micro areas (CBSAs) and all urban areas/clusters (UAs) in 2010.
# Exploration:
print(len(rel))
print(len(rel[['UA','CBSA']].drop_duplicates()), '\n')
# There are 5167 records in the file, all with paired values of UA and CBSA that are unique.

print(len(rel[['UA','CBSA']][rel['CBSA']==99999]))
print(len(rel[['UA','CBSA']][rel['CBSA']!=99999]),'\n')
# There are 1038 records with a valid UA code but NO metro/micro area (CBSA).
# There are 4129 UAs with a valid CBSA.

print(len(rel[['UA','CBSA']][rel['UA']==99999]))
print(len(rel[['UA','CBSA']][rel['UA']!=99999]),'\n')
# There are 955 records with a valid CBSA code but NO urban area/cluster (UA).
# There are 4212 CBSAs with a valid UA.

print(len(rel[(rel['CBSA']!=99999) & (rel['UA']!=99999)]))
# There are 3174 records with valid unique combinations of the CBSA and UA values.

print(len(rel[(rel['CBSA']==99999) & (rel['UA']==99999)]))
# By definition there are no cases in which both the CBSA and UA are missing.

5167
5167 

1038
4129 

955
4212 

3174
0


In [9]:
print(len(rel['CBSA'].drop_duplicates()))
print(len(rel['UA'].drop_duplicates()))

956
3602


In [10]:
# ----------------------------

In [11]:
# The above means that on the average there are is least a part of more than 3 UA's for each CBSA. Thus the
# only way to get a one-to-one mapping of CBSA to UA will be to do the shapely 'within' comparison; determine
# whether the point coordinates of an IG enterprise fall within the polygon coordinates of a particular UA.

# But to create a simple 'rural_Census' flag we need only to discover whether the CBSA Code of the IG record
# matches any CBSA code in the relationship file that is not in an urban area.

In [12]:
# Is a UA in a CBSA 1) partly, 2) entirely, or 3) not at all?
# Make a dict with the UA as the key and the value a list of all CBSA codes.

In [13]:
# The uas dict has a list of CBSAs for each UA. The ua_keys dict tags each UA as rural, partly rural, or
# urban. But in InfoGroup we have only CBSA Code. We can determine if the IG record's CBSA Code matches to a UA that 
# is entirely urban and that is completely within the CBSA; i.e., it has only the IG record's CBS code in its
# CBSA list. And if it matches to a UA that is entirely rural; i.e., the UA has only a single 99999 in its CBSA list.

# The cbsas dict is a reverse version of uas: a key for every CBSA and, as the value, the list of UAs associated
# with the key. The cbsas_keys dict substitutes the urban/rural tag of each UA for its code value. Using those 
# lists of text values we can determine whethera record is rural, partly rural, or urban.

In [14]:
# UA key

In [15]:
# Create dict from list of lists.
uas = {}
for l in rel[['UA','CBSA']].values.tolist():
    try:
        uas[l[0]].append(l[1])
    except:
        uas[l[0]] = [l[1]]

In [16]:
# Validate that 99999 can occur only once in any value list in the uas dict.
n = 0
m = 0
for k,v in uas.items():
    if v.count(99999) > 1:
        n += 1
    elif v.count(99999) == 1:
        m += 1
        
print(n,m)

0 1038


In [17]:
# Categorize all the UAs as urban, rural, partly rural, or unknown.
# Partly rural == UA with some valid CBSAs and at least 1 99999;
# Rural (entirely rural) == UA with no valid CBSAs, just some 99999s;
# Urban (not at all rural) == UA with some valid CBSAs but no 99999s.
# Unknown == UA 99999 (missing)

In [18]:
def all_nines(li):
    length = len(li)
    n = 0
    for item in li:
        if item == 99999:
            n += 1
    if n == length:
        return True
    return False

In [19]:
ua_keys = dict.fromkeys(uas.keys())
for k,v in uas.items():
    if len(v) > 1:
        if all_nines(v):
            ua_keys[k] = 'rural-multi'
        elif 99999 in v:
            ua_keys[k] = 'partly rural'
        elif k == 99999:
            ua_keys[k] = 'unknown'
        else:
            ua_keys[k]= 'urban-multi'
    else:
        if v[0] == 99999:
            ua_keys[k] = 'rural-single'
        else:
            ua_keys[k] = 'urban-single'

In [20]:
# Validate that all keys have values [and remove the key for UA 99999].
for i,v in ua_keys.items():
    if v == None: 
        print(i)
print(ua_keys[99999])

# No, don't delete the key for 99999
#del ua_keys[99999]
#try:
#    print(ua_keys[99999])
#except KeyError:
#    print('No such key')

unknown


In [21]:
len_att = len(ua_keys)
entire = 0
urban = 0
part = 0
unk = 0
for k,v in ua_keys.items():
    if v in ['rural-single','rural-multi']:
        entire += 1
    elif v == 'partly rural':
        part += 1
    elif v in ['urban-single','urban-multi']:
        urban += 1
    elif v == 'unknown':
        unk += 1

print('Out of',str(len_att),'UAs:','\n',str(entire),'are entirely rural,','\n',str(part),'are partly rural,','\n',
      str(urban),'are urban, and','\n',str(unk),'is unknown.')

Out of 3602 UAs: 
 908 are entirely rural, 
 130 are partly rural, 
 2563 are urban, and 
 1 is unknown.


In [22]:
print(str((2563/3602)*100))
print(str((908/3602)*100))
print(str((130/3602)*100))
print(str((1/3602)*100))

71.15491393670183
25.2082176568573
3.6091060521932263
0.027762354247640203


In [23]:
# CBSA key
# a CBSA is not covered entirely by UAs, so is it valid to assess the rurality of a record if it only has a
# CBSA Code and no way to categorize the parts of the CBSA that are not covered by any UA?

In [24]:
# Create dict from list of lists.
cbsas = {}
for l in rel[['UA','CBSA']].values.tolist():
    try:
        cbsas[l[1]].append(l[0])
    except:
        cbsas[l[1]] = [l[0]]

In [25]:
# function to remove all occurrences of a particular value from a list.
def remove_all(li,val):
    i = 0
    while i < len(li):
        try:
            li.remove(val)
        except:
            pass
        i += 1
    return li

In [26]:
del cbsas[99999]
for k,v in cbsas.items():
    cbsas[k] = remove_all(v,99999)

In [27]:
len(cbsas)

955

In [28]:
# Find the list of UAs that are associated with a single CBSA code.
def find_uas(cbsa): 
    uas_list = []
    for k in uas.keys():
        #if k == 99999: 
            #continue
        if cbsa in uas[k]:
            uas_list.append(k)
    return uas_list

In [29]:
# Get a list of the urban/rural descriptions for each UA in a list of UAs.
def find_descs(uas_list):
    descs_list = []
    for ua in uas_list:
        descs_list.append(ua_keys[ua])
    return descs_list

In [30]:
# Object:
#       cbsa_keys: a dict with CBSA codes as keys and empty lists to be filled in with urban/rural text strings as values.

cbsa_keys = dict.fromkeys(cbsas.keys())

# Data: 
#       uas: a dict with UA codes as keys and lists of constituent CBSAs as values.
#       uas_keys: a dict with UA codes as keys and one of 'rural','partly rural', and 'urban' as a value.
#       cbsas: a dict with CBSA values as keys and lists of constituent UAs as values.

for cbsa in cbsa_keys.keys():
    li = find_uas(cbsa)
    cbsa_keys[cbsa] = find_descs(li)

In [31]:
# Validation
print(cbsas[10020])
print(cbsa_keys[10020])
print(ua_keys[37],ua_keys[43993],ua_keys[46045],ua_keys[99999])

[37, 43993, 46045]
['urban-multi', 'urban-single', 'urban-multi', 'unknown']
urban-multi urban-single urban-multi unknown


In [32]:
# Validation
print(cbsas[48140])
print(cbsa_keys[48140])
print(ua_keys[91],ua_keys[55144],ua_keys[93025],ua_keys[99999])

[91, 55144, 93025]
['partly rural', 'urban-multi', 'urban-single', 'unknown']
partly rural urban-multi urban-single unknown


In [33]:
# Now add an 'invalid_rural_Census' variable with appropriate values(?). Invalid because the logic is
# faulty, as it turns out. For example...
# If one description is 'partly rural', the whole CBSA is partly rural
# If there is a mixture of urban and rural (and partly rural), the whole CBSA is partly rural
# Otherwise, rural and urban are both all rural or all urban
def assign_text(code):
    try:
        li = cbsa_keys[code]
    except KeyError:
        return 'not available'
    
    if 'partly rural' in li:
        return 'partly rural'
    elif 'urban' in li and 'rural' in li:
        return 'partly rural'
    elif 'rural' in li:
        return 'rural'
    elif 'urban' in li:
        return 'urban'
    else:
        return 'unknown'

In [34]:
df['invalid_rural_Census'] = df['CBSA Code'].apply(assign_text)

In [35]:
df['invalid_rural_Census'].value_counts()

unknown          10099729
partly rural      2942686
not available     1691022
Name: invalid_rural_Census, dtype: int64

In [36]:
df['invalid_rural_Census'].value_counts(normalize=True) * 100

unknown          68.549715
partly rural     19.972841
not available    11.477444
Name: invalid_rural_Census, dtype: float64

In [37]:
# No cases in which all UAs for a CBSA are rural?
n = 0
for i,v in cbsa_keys.items():
    if v.count('rural') == len(v):
        n += 1
n

0

In [38]:
# Are all the 'not availables' KeyErrors? Yes.
unknowns = set([i for i in df['CBSA Code'][df['invalid_rural_Census']=='not available']])
print(len(unknowns))

n = 0
master = set(cbsas.keys())
for unk in unknowns:
    if unk in master:
        n += 1
print(n)

# Thus 53 CBSA Codes in the InfoGroup file are not listed in the relationship file. These are the CBSA Code
# values for 1,691,022 records.

54
0


In [39]:
# Simple 'reverse-rurality' indicator: for each CBSA in the InfoGroup file, does it match to any CBSA in the 
# relationship file that is matched there to a valid UA code?

In [40]:
pairs = rel[['UA','CBSA']].values.tolist()
print(len(pairs))
urban_cbsas = []
for pair in pairs:
    if pair[0] == 99999 or pair[1] == 99999:
        continue
    else:
        urban_cbsas.append(pair[1])
print(len(urban_cbsas))
urban_cbsas_set = set(urban_cbsas)
print(len(urban_cbsas_set))

5167
3174
955


In [41]:
def isit_rural(code):
    if code in urban_cbsas_set:
        return 0
    else:
        return 1

In [42]:
df['rural_Census_general'] = df['CBSA Code'].apply(isit_rural)

In [43]:
print(df['rural_Census_general'].value_counts())
print(df['rural_Census_general'].value_counts(normalize=True) * 100)

0    13042415
1     1691022
Name: rural_Census_general, dtype: int64
0    88.522556
1    11.477444
Name: rural_Census_general, dtype: float64


In [44]:
df.drop('invalid_rural_Census',axis=1,inplace=True)

In [45]:
# ---------------------------------

In [46]:
# For later analysis we will want the six-digit NAICS codes as well as the 8-digit code that comes with
# InfoGroup and the 2-digit code and description that we created in the #1 notebook.
# We'll also include some descriptions for some codes that the project statement mentions specifically as
# definitely agricultural.

df['NAICS6'] = df['Primary NAICS Code'].apply(lambda x: str(x)[:6])

In [47]:
ag_naics = {'112112':'Cattle Feedlots',
            '115111':'Cotton Ginning',
            '115112':'Soil Preparation, Planting and Cultivating',
            '115113':'Crop Harvesting, Primarily by Machine',
            '115114':'Postharvest Crop Activities except Cotton Ginning',
            '115115':'Farm Labor Contractors and Crew Leaders',
            '115116':'Farm Management Services',
            '423820':'Farm and Garden Machinery and Equipment Merchant Wholesalers',
            '424430':'Dairy Product Merchant Wholesalers',
            '424440':'Poultry and Poultry Product Merchant Wholesalers',
            '424480':'Fresh Fruit and Vegetable Merchant Wholesalers',
            '424510':'Grain and Field Bean Merchant Wholesalers',
            '424520':'Livestock Merchant Wholesalers',
            '424590':'Other Farm Product Raw Material Merchant Wholesalers',
            '424910':'Farm Supplies Merchant Wholesalers',
            '493130':'Farm Product Warehousing and Storage'}

In [48]:
def naics6_desc(code):
    if code in ag_naics.keys():
        return ag_naics[code]
    return np.nan

In [49]:
df['NAICS6 desc'] = df['NAICS6'].apply(naics6_desc)

In [50]:
df['NAICS6 desc'].value_counts()

Farm Supplies Merchant Wholesalers                              18210
Farm and Garden Machinery and Equipment Merchant Wholesalers    11859
Farm Product Warehousing and Storage                             4356
Fresh Fruit and Vegetable Merchant Wholesalers                   3829
Farm Management Services                                         2353
Livestock Merchant Wholesalers                                   2199
Dairy Product Merchant Wholesalers                               1276
Soil Preparation, Planting and Cultivating                       1211
Grain and Field Bean Merchant Wholesalers                        1119
Cattle Feedlots                                                  1063
Postharvest Crop Activities except Cotton Ginning                 669
Other Farm Product Raw Material Merchant Wholesalers              649
Poultry and Poultry Product Merchant Wholesalers                  623
Cotton Ginning                                                    428
Farm Labor Contracto

In [51]:
len(df[df['NAICS6 desc'].isnull()])

14683155

In [52]:
df.columns

Index(['Company', 'Address Line 1', 'City', 'State', 'ZipCode', 'County Code',
       'Primary SIC Code', 'SIC6_Descriptions', 'Primary NAICS Code',
       'NAICS8 Descriptions', 'Employee Size (5) - Location',
       'Sales Volume (9) - Location', 'Business Status Code',
       'Industry Specific First Byte', 'Year Established', 'ABI',
       'Subsidiary Number', 'Parent Number', 'Parent Actual Employee Size',
       'Parent Actual Sales Volume', 'Census Tract', 'Census Block',
       'Latitude', 'Longitude', 'CBSA Code', 'CBSA Level', 'FIPS Code',
       'State FIPS', 'Continental', 'NAICS2', 'NAICS2 desc', 'CBSA Level desc',
       'rural_OMB', 'rural_Census_general', 'NAICS6', 'NAICS6 desc'],
      dtype='object')

In [53]:
# How to do the point-in-polygon comparison...

In [54]:
# The 15 programs in the points-in-polygons folder ran simultaneously as a brute force method of parallel 
# processing to locate the InfoGroup points in UA polygons. They incorporate the logic, maybe not the exact code,
# of the non-executable cells just above. They produced 15 csv files with the UA code for
# the UA record along with the shapely geometry and the ABI of the record.  Now combine those 15 files into 
# one and merge the result into the df dataframe.

In [55]:
%%time
pp_df = pd.DataFrame()

for seq in range(1,16):
    infile = 'points-in-polygons/data/dfx_%d.csv' % seq
    df_tmp = pd.read_csv(infile,dtype=object)
    pp_df = pd.concat([pp_df,df_tmp])

print(len(pp_df))

14733437
CPU times: user 1min 18s, sys: 4.93 s, total: 1min 22s
Wall time: 29.3 s


In [56]:
pp_df['ABI'] = pp_df['ABI'].astype(int)

In [57]:
%%time
# Merge with df
df_final = df.merge(pp_df,on='ABI',how='inner',indicator=True)
df_final.columns

CPU times: user 1min 48s, sys: 26.3 s, total: 2min 14s
Wall time: 1min 12s


Index(['Company', 'Address Line 1', 'City', 'State', 'ZipCode', 'County Code',
       'Primary SIC Code', 'SIC6_Descriptions', 'Primary NAICS Code',
       'NAICS8 Descriptions', 'Employee Size (5) - Location',
       'Sales Volume (9) - Location', 'Business Status Code',
       'Industry Specific First Byte', 'Year Established', 'ABI',
       'Subsidiary Number', 'Parent Number', 'Parent Actual Employee Size',
       'Parent Actual Sales Volume', 'Census Tract', 'Census Block',
       'Latitude', 'Longitude', 'CBSA Code', 'CBSA Level', 'FIPS Code',
       'State FIPS', 'Continental', 'NAICS2', 'NAICS2 desc', 'CBSA Level desc',
       'rural_OMB', 'rural_Census_general', 'NAICS6', 'NAICS6 desc',
       'geometry', 'UA', '_merge'],
      dtype='object')

In [58]:
df_final['_merge'].value_counts()

both          14733437
right_only           0
left_only            0
Name: _merge, dtype: int64

In [59]:
# The polygon 'geometry' is no longer necessary. Ditto for '_merge'.
df_final.drop(columns=['geometry','_merge'],inplace=True)

In [60]:
# Create the definitive 'rural_Census' variable.

In [61]:
# Informational
rural_pct = len(df_final[df_final['UA'] == '99999']) / len(df_final) * 100.0
print('Rural percentage by the most specific Census-based standard',str(rural_pct))

Rural percentage by the most specific Census-based standard 13.762579634337868


In [62]:
df_final['rural_Census'] = df_final['UA'].apply(lambda x: 1 if x == '99999' else 0)

In [63]:
df_final['rural_Census'].value_counts(normalize=True) * 100.0

0    86.23742
1    13.76258
Name: rural_Census, dtype: float64

In [64]:
# In the InfoGroup file we now have the Urban Area identifier. That can be merged with locational variables from 
# the Census shapefile for mapping. The output file here is a regular .csv file, not a shapefile, but it does have
# identifiers and coordinates for each UA and can be read into a GeoDataFrame.

In [65]:
df_final.columns

Index(['Company', 'Address Line 1', 'City', 'State', 'ZipCode', 'County Code',
       'Primary SIC Code', 'SIC6_Descriptions', 'Primary NAICS Code',
       'NAICS8 Descriptions', 'Employee Size (5) - Location',
       'Sales Volume (9) - Location', 'Business Status Code',
       'Industry Specific First Byte', 'Year Established', 'ABI',
       'Subsidiary Number', 'Parent Number', 'Parent Actual Employee Size',
       'Parent Actual Sales Volume', 'Census Tract', 'Census Block',
       'Latitude', 'Longitude', 'CBSA Code', 'CBSA Level', 'FIPS Code',
       'State FIPS', 'Continental', 'NAICS2', 'NAICS2 desc', 'CBSA Level desc',
       'rural_OMB', 'rural_Census_general', 'NAICS6', 'NAICS6 desc', 'UA',
       'rural_Census'],
      dtype='object')

In [66]:
%%time
outfile = 'data/df_%d_OMB_Census.csv' % year
df_final.to_csv(outfile,index=None)

CPU times: user 9min 54s, sys: 8.25 s, total: 10min 2s
Wall time: 10min 4s
